In [ ]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import sys
import pickle
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers as L
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split
from tqdm import tqdm

#os.environ["CUDA_VISIBLE_DEVICES"] = '1'
pd.set_option('display.max_columns', None)

# добавим корневую папку, в ней лежат все необходимые полезные функции для обработки данных
sys.path.append('../input/alfa-baseline/rnn_baseline/content/drive/MyDrive/Boosters.pro/Alpha')
sys.path.append('../input/alfa-baseline/rnn_baseline/content/drive/MyDrive/Boosters.pro/Alpha/rnn_baseline')

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
path_to_dataset = '../input/train-val-30-004/content/drive/MyDrive/Boosters.pro/Alpha/1/val_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_val = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_val

In [ ]:
path_to_dataset = '../input/train-val-30-004/content/drive/MyDrive/Boosters.pro/Alpha/1/train_buckets'
dir_with_datasets = os.listdir(path_to_dataset)
dataset_train = sorted([os.path.join(path_to_dataset, x) for x in dir_with_datasets])
dataset_train

In [ ]:
from data_generators import batches_generator, transaction_features
from tf_training import train_epoch, eval_model, inference
from training_aux import EarlyStopping

In [ ]:
with open('../input/alfa-baseline/rnn_baseline/content/drive/MyDrive/Boosters.pro/Alpha/rnn_baseline/constants/embedding_projections.pkl', 'rb') as f:
    embedding_projections = pickle.load(f)

In [ ]:
def build_transactions_rnn(transactions_cat_features, embedding_projections, product_col_name='product', 
                          rnn_units=128, classifier_units=32, optimizer=None):
    if not optimizer:
        optimizer = keras.optimizers.Adam(lr=1e-3)
        
    inputs = []
    cat_embeds = []
    
    for feature_name in transactions_cat_features:
        inp = L.Input(shape=(None, ), dtype='uint32', name=f'input_{feature_name}')
        inputs.append(inp)
        source_size, projection = embedding_projections[feature_name]
        emb = L.Embedding(source_size+1, projection, trainable=True, mask_zero=False, name=f'embedding_{feature_name}')(inp)
        cat_embeds.append(emb)
    
    # product feature
    inp = L.Input(shape=(1, ), dtype='uint32', name=f'input_product')
    inputs.append(inp)
    source_size, projection = embedding_projections['product']
    product_emb = L.Embedding(source_size+1, projection, trainable=True, mask_zero=False, name=f'embedding_product')(inp)
    product_emb_reshape = L.Reshape((projection, ))(product_emb)
    
    concated_cat_embeds = L.concatenate(cat_embeds)
    dropout_embeds = L.SpatialDropout1D(0.05)(concated_cat_embeds)
    
    sequences = L.Bidirectional(L.GRU(units=rnn_units, return_sequences=True))(dropout_embeds)
    
    pooled_avg_sequences = L.GlobalAveragePooling1D()(sequences)
    pooled_max_sequences = L.GlobalMaxPooling1D()(sequences)
    
    concated = L.concatenate([pooled_avg_sequences, pooled_max_sequences, product_emb_reshape])
    
    dense_intermediate = L.Dense(classifier_units, activation='relu', 
                                 kernel_regularizer=keras.regularizers.L1L2(1e-7, 1e-5))(concated)
    
    proba = L.Dense(1, activation='sigmoid')(dense_intermediate)
    
    model = Model(inputs=inputs, outputs=proba)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    return model

In [ ]:
! rm -r /content/drive/MyDrive/Boosters.pro/Alpha/rnn_baseline/checkpoints/tf_advanced_baseline
! mkdir /content/drive/MyDrive/Boosters.pro/Alpha/rnn_baseline/checkpoints/tf_advanced_baseline

In [ ]:
path_to_checkpoints = './'
es = EarlyStopping(patience=3, mode='max', verbose=True, save_path=os.path.join(path_to_checkpoints, 'best_checkpoint.pt'), 
                   metric_name='ROC-AUC', save_format='tf')

In [ ]:
num_epochs = 20
train_batch_size = 256
val_batch_size = 256

In [ ]:
model = build_transactions_rnn(transaction_features, embedding_projections, classifier_units=128)

In [ ]:
model.summary()

In [ ]:
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch+1}')
    train_epoch(model, dataset_train, batch_size=train_batch_size, shuffle=True, cur_epoch=epoch, 
                steps_per_epoch=7250)
    
    val_roc_auc = eval_model(model, dataset_val, batch_size=val_batch_size)
    model.save_weights(os.path.join(path_to_checkpoints, f'epoch_{epoch+1}_val_{val_roc_auc:.3f}.hdf5'))
    
    es(val_roc_auc, model)
    
    train_roc_auc = eval_model(model, dataset_train, batch_size=val_batch_size)
    print(f'Epoch {epoch+1} completed. Train roc-auc: {train_roc_auc}, Val roc-auc: {val_roc_auc}')
    
    if es.early_stop:
        print('Early stopping reached. Stop training...')
        break

In [ ]:
test_frame = pd.read_csv('../input/alfabattle-20/alfabattle2_test_target_contest.csv')
test_frame.head()

In [ ]:
path_to_test_dataset = '../input/alfa-baseline/test/content/drive/MyDrive/Boosters.pro/Alpha/test_buckets'
dir_with_test_datasets = os.listdir(path_to_test_dataset)
dataset_test = sorted([os.path.join(path_to_test_dataset, x) for x in dir_with_test_datasets])

dataset_test

In [ ]:
! ls $path_to_checkpoints

In [ ]:
model.load_weights(os.path.join('./epoch_6_val_0.795.hdf5'))

In [ ]:
test_preds = inference(model, dataset_test, batch_size=128)
#ТЕМА ЛАЛКА

In [ ]:
test_preds.head()

In [ ]:
test_preds.to_csv('part1_best_checkpoint.csv', index=None) # ~ 0.760 на public test
#ТЕМА ЛАЛКА